In [1]:
from mostlyai import MostlyAI
import pandas as pd
from tsl.datasets import AirQuality, MetrLA, PemsBay
from sklearn.model_selection import train_test_split
from Metrics import evaluate_datasets

TeachForcing_path = '/Users/paulmagos/Documents/TSGen/Datasets/TeachForcingDatasets/'

In [2]:
df_PemsBay = PemsBay()
df_PemsBay = df_PemsBay.dataframe()[-1000:].reset_index(drop=True)

df_MetrLA = MetrLA()
df_MetrLA = df_MetrLA.dataframe()[-1000:].reset_index(drop=True)

df_AirQuality = AirQuality(impute_nans=True, small=True)
df_AirQuality = df_AirQuality.dataframe()[-1000:].reset_index(drop=True)

df_PemsBay['gen'] = 1
df_MetrLA['gen'] = 1
df_AirQuality['gen'] = 1

df_PemsBay.columns = df_PemsBay.columns.droplevel('channels')
df_MetrLA.columns = df_MetrLA.columns.droplevel('channels')
df_AirQuality.columns = df_AirQuality.columns.droplevel('channels')

/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/tsl/datasets/pems_bay.py:84: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/tsl/datasets/pems_bay.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0, inplace=True)
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/tsl/datasets/metr_la.py:98: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/tsl/datasets/metr_la.py:109: FutureWarning: The 'method' keyword in Da

In [3]:
# split into training and validation
df_Pems_trn, df_Pems_hol = train_test_split(df_PemsBay, test_size=0.2, random_state=1)

print(f'training data with {df_Pems_trn.shape[0]:,} records and {df_Pems_trn.shape[1]} attributes')
print(f'holdout data with {df_Pems_hol.shape[0]:,} records and {df_Pems_hol.shape[1]} attributes')


df_Metr_trn, df_Metr_hol = train_test_split(df_MetrLA, test_size=0.2, random_state=1)

print(f'training data with {df_Metr_trn.shape[0]:,} records and {df_Metr_trn.shape[1]} attributes')
print(f'holdout data with {df_Metr_hol.shape[0]:,} records and {df_Metr_hol.shape[1]} attributes')

df_Air_trn, df_Air_hol = train_test_split(df_AirQuality, test_size=0.2, random_state=1)

print(f'training data with {df_Air_trn.shape[0]:,} records and {df_Air_trn.shape[1]} attributes')
print(f'holdout data with {df_Air_hol.shape[0]:,} records and {df_Air_hol.shape[1]} attributes')

training data with 800 records and 326 attributes
holdout data with 200 records and 326 attributes
training data with 800 records and 208 attributes
holdout data with 200 records and 208 attributes
training data with 800 records and 37 attributes
holdout data with 200 records and 37 attributes


In [4]:
syn_MetrLA_GTM = pd.read_csv(f'{TeachForcing_path}TeachForcingMetrLAGTM.csv')
syn_PemsBay_GTM = pd.read_csv(f'{TeachForcing_path}TeachForcingPemsBayGTM.csv')
syn_AirQuality_GTM = pd.read_csv(f'{TeachForcing_path}TeachForcingAirQualityGTM.csv')

syn_MetrLA_GTM['gen'] = 0
syn_PemsBay_GTM['gen'] = 0
syn_AirQuality_GTM['gen'] = 0


syn_MetrLA_GTLSTM = pd.read_csv(f'{TeachForcing_path}TeachForcingMetrLAGTLSTM.csv')
syn_PemsBay_GTLSTM = pd.read_csv(f'{TeachForcing_path}TeachForcingPemsBayGTLSTM.csv')
syn_AirQuality_GTLSTM = pd.read_csv(f'{TeachForcing_path}TeachForcingAirQualityGTLSTM.csv')

syn_MetrLA_GTLSTM['gen'] = 0
syn_PemsBay_GTLSTM['gen'] = 0
syn_AirQuality_GTLSTM['gen'] = 0

syn_MetrLA_GRGN = pd.read_csv(f'{TeachForcing_path}TeachForcingMetrLAGRGN.csv')
syn_PemsBay_GRGN = pd.read_csv(f'{TeachForcing_path}TeachForcingPemsBayGRGN2.csv')
syn_AirQuality_GRGN = pd.read_csv(f'{TeachForcing_path}TeachForcingAirQualityGRGN.csv')

syn_MetrLA_GRGN['gen'] = 0
syn_PemsBay_GRGN['gen'] = 0
syn_AirQuality_GRGN['gen'] = 0

In [5]:
results_df = pd.DataFrame(columns=['dataset', 'model', 'mean_difference',
 'variance_difference',
 'skewness_difference',
 'kurtosis_difference',
 'ks_test',
 'ks_test_p_value',
 'wasserstein_distance',
 'js_divergence',
 'correlation_difference',
 'mmd_linear',
 'mmd_rbf',
 'mmd_poly',
 'model_accuracy',
 'model_accuracy_on_Gen',
 'model_accuracy_train_on_both',
 'model_accuracy_train_on_both_Gen'])

In [6]:
df_PemsBay.columns = [str(col) for col in df_PemsBay.columns]
df_MetrLA.columns = [str(col) for col in df_MetrLA.columns]
df_AirQuality.columns = [str(col) for col in df_AirQuality.columns]

syn_PemsBay_GTM.columns = [str(col) for col in syn_PemsBay_GTM.columns]
syn_MetrLA_GTM.columns = [str(col) for col in syn_MetrLA_GTM.columns]
syn_AirQuality_GTM.columns = [str(col) for col in syn_AirQuality_GTM.columns]

syn_AirQuality_GTLSTM.columns = [str(col) for col in syn_AirQuality_GTLSTM.columns]
syn_PemsBay_GTLSTM.columns = [str(col) for col in syn_PemsBay_GTLSTM.columns]
syn_MetrLA_GTLSTM.columns = [str(col) for col in syn_MetrLA_GTLSTM.columns]

syn_MetrLA_GRGN.columns = [str(col) for col in syn_MetrLA_GRGN.columns]
syn_PemsBay_GRGN.columns =  [str(col) for col in syn_PemsBay_GRGN.columns]
syn_AirQuality_GRGN.columns =  [str(col) for col in syn_AirQuality_GRGN.columns]

In [7]:
# out = evaluate_datasets(df_PemsBay, df_PemsBay, 'gen')
# out['dataset'] = "PemsBay Original"
# res = pd.DataFrame(out, ["PemsBay Original"])
# results_df = pd.concat([results_df, res], ignore_index = True)

# out = evaluate_datasets(df_MetrLA, df_MetrLA, 'gen')
# out['dataset'] = "MetrLA Original"
# res = pd.DataFrame(out, ["MetrLA Original"])
# results_df = pd.concat([results_df, res], ignore_index = True)

# out = evaluate_datasets(df_AirQuality, df_AirQuality, 'gen')
# out['dataset'] = "AirQuality Original"
# res = pd.DataFrame(out, ["AirQuality Original"])
# results_df = pd.concat([results_df, res], ignore_index = True)

In [8]:
out = evaluate_datasets(df_PemsBay, syn_PemsBay_GTM, 'gen')
out['dataset'] = "PemsBay"
out['model'] = "GTM"
res = pd.DataFrame(out, ["PemsBay GTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

out = evaluate_datasets(df_MetrLA, syn_MetrLA_GTM, 'gen')
out['dataset'] = "MetrLA"
out['model'] = "GTM"
res = pd.DataFrame(out, ["MetrLA GTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

out = evaluate_datasets(df_AirQuality, syn_AirQuality_GTM, 'gen')
out['dataset'] = "AirQuality"
out['model'] = "GTM"
res = pd.DataFrame(out, ["AirQuality GTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats = pd.concat([stats, pd.DataFrame({
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ks_results = pd.concat([
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA colu

In [9]:
out = evaluate_datasets(df_PemsBay, syn_PemsBay_GTLSTM, 'gen')
out['dataset'] = "PemsBay"
out['model'] = "GTLSTM"
res = pd.DataFrame(out, ["PemsBay GTLSTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

out = evaluate_datasets(df_MetrLA, syn_MetrLA_GTLSTM, 'gen')
out['dataset'] = "MetrLA"
out['model'] = "GTLSTM"
res = pd.DataFrame(out, ["MetrLA GTLSTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

out = evaluate_datasets(df_AirQuality, syn_AirQuality_GTLSTM, 'gen')
out['dataset'] = "AirQuality"
out['model'] = "GTLSTM"
res = pd.DataFrame(out, ["AirQuality GTLSTM"])
results_df = pd.concat([results_df, res], ignore_index = True)

/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats = pd.concat([stats, pd.DataFrame({
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ks_results = pd.concat([
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA colu

In [10]:
out = evaluate_datasets(df_PemsBay, syn_PemsBay_GRGN, 'gen')
out['dataset'] = "PemsBay"
out['model'] = "GRGN"
res = pd.DataFrame(out, ["PemsBay GRGN"])
results_df = pd.concat([results_df, res], ignore_index = True)


out = evaluate_datasets(df_MetrLA, syn_MetrLA_GRGN, 'gen')
out['dataset'] = "MetrLA"
out['model'] = "GRGN"
res = pd.DataFrame(out, ["MetrLA GRGN"])
results_df = pd.concat([results_df, res], ignore_index = True)


out = evaluate_datasets(df_AirQuality, syn_AirQuality_GRGN, 'gen')
out['dataset'] = "AirQuality"
out['model'] = "GRGN"
res = pd.DataFrame(out, ["AirQuality GRGN"])
results_df = pd.concat([results_df, res], ignore_index = True)

/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats = pd.concat([stats, pd.DataFrame({
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ks_results = pd.concat([
/Users/paulmagos/Documents/TSGen/Notebooks/Metrics.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA colu

In [11]:
results_df = results_df.sort_values(by='mmd_linear').reset_index(drop=True)
results_df[results_df['dataset'] == 'MetrLA']

,dataset,model,mean_difference,variance_difference,skewness_difference,kurtosis_difference,ks_test,ks_test_p_value,wasserstein_distance,js_divergence,...,mmd_rbf,mmd_poly,model_accuracy,model_accuracy_on_Gen,model_accuracy_train_on_both,model_accuracy_train_on_both_Gen,inception_score_mean,inception_score_std,t_stat,t_stat_p_value
3,MetrLA,GTLSTM,2.732190,114.022991,2.211501,8.180405,0.450312,1.903052e-08,6.114224,0.614248,...,0.029450,NaN,0.0,0.0,1.0,1.0,0.000000,0.000000,-0.619286,0.065245
7,MetrLA,GTM,4.323815,53.714127,1.941160,7.393603,0.320763,9.668278e-05,5.983050,0.685032,...,0.002096,NaN,0.0,0.0,1.0,1.0,0.995311,0.995314,-2.081344,0.027905
8,MetrLA,GRGN,4.704967,457.520964,2.238577,8.694813,0.392715,9.089599e-45,14.820494,0.634796,...,0.002096,NaN,0.0,0.0,1.0,1.0,0.398485,0.796973,-0.868781,0.073451


In [12]:
results_df['tmp'] = results_df['mean_difference'] + results_df['variance_difference']

In [13]:
results_df = results_df.sort_values(by=['tmp']).reset_index(drop=True)
results_df

,dataset,model,mean_difference,variance_difference,skewness_difference,kurtosis_difference,ks_test,ks_test_p_value,wasserstein_distance,js_divergence,...,mmd_poly,model_accuracy,model_accuracy_on_Gen,model_accuracy_train_on_both,model_accuracy_train_on_both_Gen,inception_score_mean,inception_score_std,t_stat,t_stat_p_value,tmp
0,MetrLA,GTM,4.323815,53.714127,1.941160,7.393603,0.320763,9.668278e-05,5.983050,0.685032,...,NaN,0.0,0.0,1.000,1.000000,0.995311,0.995314,-2.081344,0.027905,58.037942
1,PemsBay,GTLSTM,3.355599,92.338557,2.087390,7.718859,0.502320,4.443758e-07,5.882983,0.586812,...,NaN,0.0,0.0,1.000,1.000000,1.987370,0.005471,1.042225,0.046539,95.694156
2,PemsBay,GRGN,3.270937,97.717162,2.238661,8.157078,0.525289,3.215334e-13,6.147888,0.622257,...,NaN,0.0,0.0,1.000,1.000000,1.739782,0.580018,8.775906,0.035564,100.988099
3,MetrLA,GTLSTM,2.732190,114.022991,2.211501,8.180405,0.450312,1.903052e-08,6.114224,0.614248,...,NaN,0.0,0.0,1.000,1.000000,0.000000,0.000000,-0.619286,0.065245,116.755180
4,PemsBay,GTM,1.938705,171.762852,1.921164,7.496421,0.298519,6.885741e-06,6.521830,0.622777,...,NaN,0.0,0.0,1.000,1.000000,1.884787,0.016058,0.615191,0.157694,173.701557
5,MetrLA,GRGN,4.704967,457.520964,2.238577,8.694813,0.392715,9.089599e-45,14.820494,0.634796,...,NaN,0.0,0.0,1.000,1.000000,0.398485,0.796973,-0.868781,0.073451,462.225931
6,AirQuality,GTLSTM,7.111527,1046.819169,0.348541,1.959327,0.138398,5.967616e-03,10.960778,0.252239,...,NaN,0.0,0.0,0.936,0.992214,0.000000,0.000000,1.194029,0.172821,1053.930696
7,AirQuality,GTM,11.811962,1350.150414,0.343568,1.694042,0.266717,3.775109e-04,19.118943,0.378897,...,NaN,0.0,0.0,0.855,0.937709,0.180545,0.541635,-5.824113,0.090367,1361.962376
8,AirQuality,GRGN,8.110985,2399.165217,1.057422,1.955987,0.425028,3.592759e-55,33.209360,0.533192,...,NaN,0.0,0.0,0.972,1.000000,0.000000,0.000000,2.930005,0.066377,2407.276202


In [15]:
df = results_df

# Filter for models that contain "GRGN" in the dataset
# grgn_models = df[df['dataset'].str.contains("GRGN")]
grgn_models = results_df
# Metrics to focus on
metrics_to_compare = ['mean_difference', 'variance_difference', 'ks_test', 'wasserstein_distance', 
                      'js_divergence', 'correlation_difference', 'model_accuracy', 'inception_score_mean']

# Find the minimum for distance-based metrics and the maximum for accuracy/inception-based metrics
best_grgn_models = grgn_models.loc[:, ['dataset', 'model'] + metrics_to_compare].copy()

# For distance-based metrics, lower is better, so we will look for the minimum values.
# For accuracy-based and inception score-based metrics, higher is better.

# Normalize each metric: for distance-based (lower is better), for accuracy-based (higher is better)
for col in ['mean_difference', 'variance_difference', 'ks_test', 'wasserstein_distance', 
            'js_divergence', 'correlation_difference']:
    best_grgn_models[col + '_rank'] = best_grgn_models[col].rank(ascending=True)

# For accuracy and inception score, higher is better
for col in ['model_accuracy', 'inception_score_mean']:
    best_grgn_models[col + '_rank'] = best_grgn_models[col].rank(ascending=False)

# Calculate total rank
best_grgn_models['total_rank'] = best_grgn_models[[col + '_rank' for col in metrics_to_compare]].sum(axis=1)

# Sort by total rank to find the best overall GRGN model
best_grgn_models = best_grgn_models.sort_values('total_rank').reset_index(drop=True)

# Display the best GRGN model
best_grgn_models


,dataset,model,mean_difference,variance_difference,ks_test,wasserstein_distance,js_divergence,correlation_difference,model_accuracy,inception_score_mean,mean_difference_rank,variance_difference_rank,ks_test_rank,wasserstein_distance_rank,js_divergence_rank,correlation_difference_rank,model_accuracy_rank,inception_score_mean_rank,total_rank
0,MetrLA,GTM,4.323815,53.714127,0.320763,5.983050,0.685032,0.211923,0.0,0.995311,5.0,1.0,4.0,2.0,9.0,1.0,5.0,4.0,31.0
1,PemsBay,GTM,1.938705,171.762852,0.298519,6.521830,0.622777,0.310803,0.0,1.884787,1.0,5.0,3.0,5.0,7.0,4.0,5.0,2.0,32.0
2,PemsBay,GTLSTM,3.355599,92.338557,0.502320,5.882983,0.586812,0.488589,0.0,1.987370,4.0,2.0,8.0,1.0,4.0,8.0,5.0,1.0,33.0
3,AirQuality,GTLSTM,7.111527,1046.819169,0.138398,10.960778,0.252239,0.228454,0.0,0.000000,7.0,7.0,1.0,6.0,1.0,3.0,5.0,8.0,38.0
4,PemsBay,GRGN,3.270937,97.717162,0.525289,6.147888,0.622257,0.385929,0.0,1.739782,3.0,3.0,9.0,4.0,6.0,7.0,5.0,3.0,40.0
5,MetrLA,GTLSTM,2.732190,114.022991,0.450312,6.114224,0.614248,0.349951,0.0,0.000000,2.0,4.0,7.0,3.0,5.0,6.0,5.0,8.0,40.0
6,MetrLA,GRGN,4.704967,457.520964,0.392715,14.820494,0.634796,0.220906,0.0,0.398485,6.0,6.0,5.0,7.0,8.0,2.0,5.0,5.0,44.0
7,AirQuality,GTM,11.811962,1350.150414,0.266717,19.118943,0.378897,0.333919,0.0,0.180545,9.0,8.0,2.0,8.0,2.0,5.0,5.0,6.0,45.0
8,AirQuality,GRGN,8.110985,2399.165217,0.425028,33.209360,0.533192,0.735610,0.0,0.000000,8.0,9.0,6.0,9.0,3.0,9.0,5.0,8.0,57.0
